In [1]:
from joblib import load
import pandas as pd
from itertools import permutations
import pickle

filename = '../Data/list_tuplas.pkl'
lista_tuplas = open(filename, 'rb')
with open(filename, 'rb') as f:
    lista_tuplas = pickle.load(f)

df = pd.read_csv('../Data/equipos_data.csv', index_col = 0)
partidos = list(permutations(df['equipo'].unique(), 2))

model = load('trainedMLP.joblib')

C:\Users\migue\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [2]:
import pandas as pd

data = []

for fecha, finde in lista_tuplas:
    for partido in partidos:
        data.append(partido + (int(fecha[3:5]), finde, fecha))
        
df = pd.DataFrame(data, columns = ['equipo1', 'equipo2', 'mes', 'fin_de_semana', 'fecha'])
df.astype({'mes': 'object'}).dtypes

historico = pd.get_dummies(df.drop('fecha', axis = 1), prefix = ['equipo1','equipo2','mes'], columns = ['equipo1','equipo2','mes'])

In [3]:
orden_original = []

with open('columnas.txt', encoding='utf-8') as fp:
    for fila in fp:
        orden_original.append(fila[:-1])
        
orden_original.pop(1)
all(historico.columns == orden_original)

True

In [18]:
predicciones = {}
df['pred'] = model.predict(historico)

for i, fila in df.iterrows():
    predicciones[(fila['equipo1'], fila['equipo2']), fila['fecha']] = fila['pred']
    
with open('../Data/predicciones.pickle', 'wb') as file:
    pickle.dump(predicciones, file)

C:\Users\migue\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
